# **FGV EESP**

- ## Dados Econômicos - Brasil
---



### **Bibliotecas**

In [ ]:
# Obter os dados do SGS do Banco Central do Brasil
install.packages("rbcb")

# Salvar dados em formato .xlsx
install.packages("writexl")

# Implementar filtro HP
install.packages("mFilter")

In [ ]:
library(dplyr)
library(httr)
library(jsonlite)
library(lubridate)
library(mFilter)
library(readxl)
library(rbcb)
library(tidyverse)
library(writexl)
library(zoo)

### **Dados**

- #### **Expectativa de Inflação do Mercado**

  * Referência: [Olinda BCB](https://olinda.bcb.gov.br/olinda/servico/Expectativas/versao/v1/aplicacao#!/recursos/ExpectativasMercadoInflacao12Meses#eyJmb3JtdWxhcmlvIjp7IiRmb3JtYXQiOiJqc29uIiwiJHRvcCI6MTAwLCIkZmlsdGVyIjoiBDkEIGVxIDAgYW5kIAQyBCBlcSAnUycgYW5kIAQwBCBlcSAnSVBDQScgYW5kIAQxBCBnZSAnMjAwNi0wMS0wMScgYW5kIAQxBCBsZSAnMjAyMy0wNi0zMCciLCIkb3JkZXJieSI6IgQxBCBkZXNjIn0sInByb3ByaWVkYWRlcyI6WzAsMSwzLDQsNV0sInBlc3F1aXNhZG8iOnRydWUsImFjdGl2ZVRhYiI6InRhYmxlIiwiZ3JpZFN0YXRlIjp7AzADOlt7A0IDIgQwBCIsA0EDfSx7A0IDIgQxBCIsA0EDfSx7A0IDIgQzBCIsA0EDfSx7A0IDIgQzBG5hIiwDQQN9LHsDQgMiBDUEIiwDQQN9XSwDMQM6e30sAzIDOltdLAMzAzp7fSwDNAM6e30sAzUDOnt9fSwicGl2b3RPcHRpb25zIjp7A2EDOnt9LANiAzpbXSwDYwM6NTAwLANkAzpbIgQxBCJdLANlAzpbXSwDZgM6WyJN6WRpYSJdLANnAzoidmFsdWVfel90b19hIiwDaAM6InZhbHVlX3pfdG9fYSIsA2kDOnt9LANqAzp7fSwDawM6ODUsA2wDOmZhbHNlLANtAzp7fSwDbgM6e30sA28DOiJN6WRpYSIsA3ADOiJMaW5lIENoYXJ0In19)

In [3]:
# Definir a URL da API do BCB para obter dados de Expectativa do Mercado para IPCA nos próximos 12 meses
url <- "https://olinda.bcb.gov.br/olinda/servico/Expectativas/versao/v1/odata/ExpectativasMercadoInflacao12Meses?$filter=baseCalculo%20eq%200%20and%20Suavizada%20eq%20'S'%20and%20Indicador%20eq%20'IPCA'%20and%20Data%20ge%20'2005-12-01'%20and%20Data%20le%20'2023-06-30'&$orderby=Data%20desc&$format=json&$select=Indicador,Data,Media,Mediana,DesvioPadrao"

# Realizar a consulta e ler o arquivo JSON
response <- GET(url)
data_json <- rawToChar(response$content)

# Ajuste das colunas de data
focus_inflation_expectation <- fromJSON(data_json)$value %>%
  mutate(day = ymd(Data),
         month_date = floor_date(day, unit = "month"),
         year = year(month_date),
         day_of_week = weekdays(day),
         month_name = month(month_date, label = TRUE, abbr = TRUE))

# Filtrar apenas as sextas-feiras (Referência do Boletim Focus)
df_fridays_only <- focus_inflation_expectation %>%
  filter(day_of_week == "Friday")

# Calcular média da coluna "Mediana" agrupada por "year" e "month_date"
df_olinda <- as.data.frame(df_fridays_only %>%
  group_by(year, month_date, month_name) %>%
  summarize(market_inflation_expectation = round(mean(Mediana), 2)))

# Converter variável em série temporal
df_olinda$market_inflation_expectation <- ts(df_olinda$market_inflation_expectation, start = c(2005, 12), frequency = 12)

# Adicionar a coluna 'month' com base em 'market_inflation_expectation'
df_olinda$month <- time(df_olinda$market_inflation_expectation)

`summarise()` has grouped output by 'year', 'month_date'. You can override
using the `.groups` argument.


- #### **Meta de Inflação**

  * Referência: [BCB](https://www.bcb.gov.br/controleinflacao/historicometas)

In [4]:
# Histórico das metas para a inflação

# Caminho do arquivo original
file_path <- "https://github.com/matheusrmorgado/inflation-expectation/raw/main/data/bcb_inflation_target.xlsx"

# Download do arquivo
download.file(file_path, "bcb_inflation_target.xlsx")

# Ler o arquivo Excel e converter em dataframe
df_bcb <- as.data.frame(read_excel("bcb_inflation_target.xlsx"))

# Filtrar os dados de 2005-12-01 até 2023-06-01
df_bcb <- df_bcb %>% filter(as.Date(month_date) >= as.Date("2005-12-01") & as.Date(month_date) <= as.Date("2023-06-01"))

# Lista de variáveis
variables_to_convert <- c("inflation_target","inflation_target_lower_bound","inflation_target_upper_bound","inflation_target_wtd_avg","inflation_target_lower_bound_wtd_avg","inflation_target_upper_bound_wtd_avg")

# Loop para converter as variáveis em séries temporais
for (variable in variables_to_convert) {
  df_bcb[[variable]] <- ts(df_bcb[[variable]], start = c(2005, 12), frequency = 12)
}

# Selecionar apenas variáveis relacionadas a inflação
df_bcb <- df_bcb[, 3:8]

- #### **Expectativa de Inflação dos Consumidores**

  * Referência: [FGV IBRE](https://extra-ibre.fgv.br/IBRE/sitefgvdados/default.aspx)

In [5]:
# Indicador de Expectativa de Inflação dos Consumidores
# Código FGV Dados: 1416188

# Caminho do arquivo original
file_path <- "https://github.com/matheusrmorgado/inflation-expectation/raw/main/data/fgv_ibre.xlsx"

# Download do arquivo
download.file(file_path, "fgv_ibre.xlsx")

# Ler o arquivo Excel e converter em dataframe
df_fgv <- as.data.frame(read_excel("fgv_ibre.xlsx"))

# Filtrar os dados de 2005-12-01 até 2023-06-01
df_fgv <- df_fgv %>% filter(as.Date(month_date) >= as.Date("2005-12-01") & as.Date(month_date) <= as.Date("2023-06-01"))

# Conversão em formato timeseries
df_consumer <- as.data.frame(ts(df_fgv$consumer_inflation_expectation, start = c(2005, 12), frequency = 12))
df_consumer <- rename(df_consumer, consumer_inflation_expectation = x)

- #### **Dados Macroeconômicos**

  * Referência: [SGS BCB](https://www3.bcb.gov.br/sgspub/localizarseries/localizarSeries.do?method=prepararTelaLocalizarSeries)

In [6]:
# Índice nacional de preços ao consumidor amplo acumulado em 12 meses (%)
# SGS: 13522
df_ipca <- as.data.frame(get_series(13522, start_date = "2005-12-01", end_date = "2023-06-01", as = "ts")) %>% rename(ipca = x)

# Taxa de juros Selic acumulada no mês anualizada base 252 (% a.a.)
# SGS: 4189
df_selic <- as.data.frame(get_series(4189, start_date = "2005-12-01", end_date = "2023-06-01", as = "ts")) %>% rename(selic = x)

# Selic deflacionada pelo IPCA
df_selic_deflated <- 100*(((1+df_selic/100)/(1+df_ipca/100))-1) %>% rename(selic_deflated = selic)

# Índice da taxa de câmbio real efetiva (IPCA) - Jun/1994=100
# SGS: 11752
df_real_exchange_rate <- as.data.frame(get_series(11752, start_date = "2005-12-01", end_date = "2023-06-01", as = "ts")) %>% rename(real_exchange_rate = x)
df_real_exchange_rate$real_exchange_rate_trend <- as.numeric(hpfilter(df_real_exchange_rate$real_exchange_rate)$trend)
df_real_exchange_rate$real_exchange_rate_trend <- ts(df_real_exchange_rate$real_exchange_rate_trend, start = c(2005, 12), frequency = 12)
df_real_exchange_rate$real_exchange_rate_trend_diff <- 100*((df_real_exchange_rate$real_exchange_rate/df_real_exchange_rate$real_exchange_rate_trend) - 1)

# PIB acumulado dos últimos 12 meses - Valores correntes - R$ (bilhões)
# SGS: 4382
df_pib <- as.data.frame(get_series(4382, start_date = "2005-12-01", end_date = "2023-06-01", as = "ts")) %>% rename(pib = x)

# Necessidade de Financiamento do Setor Público (NFSP) sem desvalorização cambial - Fluxo acumulado em 12 meses - Resultado primário - Total - Setor público consolidado
# SGS: 5078
df_primary_fiscal_result <- as.data.frame(get_series(5078, start_date = "2005-12-01", end_date = "2023-06-01", as = "ts")) %>% rename(primary_fiscal_result = x)

# Necessidade de Financiamento do Setor Público (NFSP) sem desvalorização cambial (% PIB) - Fluxo acumulado em 12 meses - Resultado primário - Total - Setor público consolidado
# SGS: 5793
df_primary_fiscal_result_pct_pib <- as.data.frame(get_series(5793, start_date = "2005-12-01", end_date = "2023-06-01", as = "ts")) %>% rename(primary_fiscal_result_pct_pib = x)

In [ ]:
# Renda Nacional Disponível Bruta das Famílias - em valores correntes (média móvel trimestral)
# SGS: 29023
df_income <- as.data.frame(get_series(29023, start_date = "2004-06-01", end_date = "2023-06-01")) %>%
  rename(household_income = "29023") %>%
  mutate(household_income = rollapply(household_income, width = 12, FUN = mean, fill = NA, align = "right")) %>%
  mutate(income_increase = 100 * ((household_income - lag(household_income, n = 12)) / lag(household_income, n = 12))) %>%
  filter(date >= as.Date("2005-12-01")) %>%
  select(income_increase)

df_income$income_increase <- ts(df_income$income_increase, start = c(2005, 12), frequency = 12)
df_income_deflated <- 100*(((1+df_income/100)/(1+df_ipca/100))-1) %>% rename(income_increase_deflated = "income_increase")

# Renda Nacional Disponível Bruta das Famílias, em valores constantes, deflacionada pelo IPCA (média móvel trimestral)
# SGS: 29025
df_rndbf_deflated <- as.data.frame(get_series(29025, start_date = "2004-06-01", end_date = "2023-06-01")) %>%
  rename(household_income_deflated = "29025") %>%
  mutate(household_income_deflated = rollapply(household_income_deflated, width = 12, FUN = mean, fill = NA, align = "right")) %>%
  filter(date >= as.Date("2005-12-01")) %>%
  select(household_income_deflated)

df_rndbf_deflated$household_income_deflated <- ts(df_rndbf_deflated$household_income_deflated, start = c(2005, 12), frequency = 12)
df_rndbf_deflated$household_income_deflated_trend <- as.numeric(hpfilter(df_rndbf_deflated$household_income_deflated)$trend)
df_rndbf_deflated$household_income_deflated_trend <- ts(df_rndbf_deflated$household_income_deflated_trend, start = c(2005, 12), frequency = 12)
df_rndbf_deflated$household_income_deflated_trend_diff <- 100*((df_rndbf_deflated$household_income_deflated/df_rndbf_deflated$household_income_deflated_trend) - 1)

In [9]:
# Salário Mínimo
# SGS: 1619
df_wage <- as.data.frame(get_series(1619, start_date = "2004-06-01", end_date = "2023-06-01")) %>%
  rename(minimum_wage = "1619") %>%
  mutate(minimum_wage = rollapply(minimum_wage, width = 12, FUN = mean, fill = NA, align = "right")) %>%
  mutate(minimum_wage_increase = 100 * ((minimum_wage - lag(minimum_wage, n = 12)) / lag(minimum_wage, n = 12))) %>%
  filter(date >= as.Date("2005-12-01")) %>%
  select(minimum_wage_increase)

df_wage$minimum_wage_increase <- ts(df_wage$minimum_wage_increase, start = c(2005, 12), frequency = 12)
df_wage_deflated <- 100*(((1+df_wage/100)/(1+df_ipca/100))-1) %>% rename(minimum_wage_increase_deflated = "minimum_wage_increase")

- #### **União de todas as bases de dados**

In [10]:
# Realiza a unificação de todas as tabelas do BCB e FGV IBRE
df_public_data <- cbind(df_olinda, df_consumer, df_bcb, df_ipca, df_selic, df_selic_deflated, df_wage, df_wage_deflated, df_income, df_income_deflated, df_rndbf_deflated, df_real_exchange_rate, df_pib, df_primary_fiscal_result, df_primary_fiscal_result_pct_pib)

# Calcula o lag para IPCA e Selic
df_public_data$ipca_lag <- c(NA, df_public_data$ipca[-nrow(df_public_data)])
df_public_data$selic_deflated_lag <- c(NA, df_public_data$selic_deflated[-nrow(df_public_data)])
df_public_data$primary_fiscal_result_pct_pib_lag <- c(NA, df_public_data$primary_fiscal_result_pct_pib[-nrow(df_public_data)])
df_public_data$real_exchange_rate_trend_diff_lag <- c(NA, df_public_data$real_exchange_rate_trend_diff[-nrow(df_public_data)])
df_public_data$household_income_deflated_trend_diff_lag <- c(NA, df_public_data$household_income_deflated_trend_diff[-nrow(df_public_data)])

# Converter as colunas "ipca_lag" e "selic_lag" para séries temporais
df_public_data$ipca_lag <- ts(df_public_data$ipca_lag, start = c(2005, 12), frequency = 12)
df_public_data$selic_deflated_lag <- ts(df_public_data$selic_deflated_lag, start = c(2005, 12), frequency = 12)
df_public_data$primary_fiscal_result_pct_pib_lag <- ts(df_public_data$primary_fiscal_result_pct_pib_lag, start = c(2005, 12), frequency = 12)
df_public_data$real_exchange_rate_trend_diff_lag <- ts(df_public_data$real_exchange_rate_trend_diff_lag, start = c(2005, 12), frequency = 12)
df_public_data$household_income_deflated_trend_diff_lag <- ts(df_public_data$household_income_deflated_trend_diff_lag, start = c(2005, 12), frequency = 12)

# Reorganiza as colunas, colocando "month_date" e "month" nas primeiras posições
df_public_data <- df_public_data %>% dplyr::select(year, month_name, month_date, month, everything())

In [11]:
# Identificar as variáveis que são Time-Series
ts_cols <- sapply(df_public_data, is.ts)

# Filtrar o dataframe
df_public_data <- df_public_data %>%
  filter(month_date >= as.Date("2006-01-01"))

# Transformar as colunas identificadas como Time-Series de volta em Time-Series
df_public_data[ts_cols] <- lapply(df_public_data[ts_cols], function(x) ts(x, start = c(2006, 1), frequency = 12))

- #### **Visualização da base de dados**

In [12]:
str(df_public_data)

'data.frame':	210 obs. of  33 variables:
 $ year                                    : num  2006 2006 2006 2006 2006 ...
 $ month_name                              : Ord.factor w/ 12 levels "Jan"<"Feb"<"Mar"<..: 1 2 3 4 5 6 7 8 9 10 ...
 $ month_date                              : Date, format: "2006-01-01" "2006-02-01" ...
 $ month                                   : Time-Series  from 2006 to 2023: 2006 2006 2006 2006 2006 ...
 $ market_inflation_expectation            : Time-Series  from 2006 to 2023: 4.58 4.49 4.36 4.16 4.18 4.25 4.4 4.48 4.29 4.05 ...
 $ consumer_inflation_expectation          : Time-Series  from 2006 to 2023: 6.9 7 6.9 6.5 6.7 6.3 6.9 6.4 6.3 5.8 ...
 $ inflation_target                        : Time-Series  from 2006 to 2023: 4.5 4.5 4.5 4.5 4.5 4.5 4.5 4.5 4.5 4.5 ...
 $ inflation_target_lower_bound            : Time-Series  from 2006 to 2023: 2.5 2.5 2.5 2.5 2.5 2.5 2.5 2.5 2.5 2.5 ...
 $ inflation_target_upper_bound            : Time-Series  from 2006 to 2023: 6

In [13]:
head(df_public_data, 5)

,year,month_name,month_date,month,market_inflation_expectation,consumer_inflation_expectation,inflation_target,inflation_target_lower_bound,inflation_target_upper_bound,inflation_target_wtd_avg,⋯,real_exchange_rate_trend,real_exchange_rate_trend_diff,pib,primary_fiscal_result,primary_fiscal_result_pct_pib,ipca_lag,selic_deflated_lag,primary_fiscal_result_pct_pib_lag,real_exchange_rate_trend_diff_lag,household_income_deflated_trend_diff_lag
,<dbl>,<ord>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,2006,Jan,2006-01-01,2006.000,4.58,6.9,4.5,2.5,6.5,4.5,⋯,98.40322,2.313721,2190643,-74923.44,-3.42,5.69,11.87435,-3.74,2.147822,2.754730
2,2006,Feb,2006-02-01,2006.083,4.49,7.0,4.5,2.5,6.5,4.5,⋯,98.02805,-2.640109,2207671,-74569.50,-3.38,5.70,11.30558,-3.42,2.313721,2.488664
3,2006,Mar,2006-03-01,2006.167,4.36,6.9,4.5,2.5,6.5,4.5,⋯,97.65292,-2.419715,2225145,-73223.35,-3.29,5.51,11.15534,-3.38,-2.640109,2.064065
4,2006,Apr,2006-04-01,2006.250,4.16,6.5,4.5,2.5,6.5,4.5,⋯,97.27786,-2.197685,2235813,-74002.58,-3.31,5.32,10.84314,-3.29,-2.419715,1.706890
5,2006,May,2006-05-01,2006.333,4.18,6.7,4.5,2.5,6.5,4.5,⋯,96.90286,2.484076,2256419,-74308.97,-3.29,4.63,11.04846,-3.31,-2.197685,1.268233


- #### **Salvar em formato `.csv` e `.xlsx`**

In [14]:
# Salve o DataFrame como um arquivo CSV localmente
write.csv(df_public_data, file = "public_data.csv", row.names = FALSE)

In [15]:
# Salve o DataFrame como um arquivo XLSX localmente
write_xlsx(df_public_data, path = "/content/public_data.xlsx")